In [1]:
import mdtraj as md
import bpmfwfft.grids as grids
import numpy as np

In [2]:
from os import walk
import socket
if socket.gethostname() == 'jim-Mint':
    ppi_path = '/media/jim/Research_TWO/FFT_PPI'
else:
    ppi_path = '/home/jtufts/Desktop/FFT_PPI'

In [3]:
f = {}
for (dirpath, dirnames, filenames) in walk(f'{ppi_path}/2.redock/2.minimize'):
    if filenames[0] != 'run_me.sh':
        if 'complex.pdb' in filenames:
            f[dirpath.split('/')[7]] = dirpath + '/complex.pdb'

In [5]:
import netCDF4 as nc
import numpy as np

grid_path = f'{ppi_path}/2.redock/4.receptor_grid/2OOB_A:B'
grid_nc = '%s/grid.nc'%grid_path


#parsing some of the netcdf variables from grid.nc
grid_variables = nc.Dataset(grid_nc, 'r').variables
counts = nc.Dataset(grid_nc, 'r').variables["counts"][:]
x = nc.Dataset(grid_nc, 'r').variables["x"][:]
y = nc.Dataset(grid_nc, 'r').variables["y"][:]
z = nc.Dataset(grid_nc, 'r').variables["z"][:]
electrostatic = nc.Dataset(grid_nc, 'r').variables["electrostatic"][:]
lja = nc.Dataset(grid_nc, 'r').variables["LJa"][:]
ljr = nc.Dataset(grid_nc, 'r').variables["LJr"][:]
sasai = nc.Dataset(grid_nc, 'r').variables["SASAi"][:]
sasar = nc.Dataset(grid_nc, 'r').variables["SASAr"][:]
trans_crd = nc.Dataset(grid_nc, 'r').variables["trans_crd"][:]

In [3]:
from bpmfwfft.grids import RecGrid
from bpmfwfft.grids import LigGrid

rec_prmtop = f"{ppi_path}/2.redock/1.amber/2OOB_A:B/receptor.prmtop"
lj_sigma_scal_fact = 1.0
rec_inpcrd = f"{ppi_path}/2.redock/1.amber/2OOB_A:B/receptor.inpcrd"

bsite_file = None
grid_nc_file = f"{ppi_path}/2.redock/4.receptor_grid/2OOB_A:B/grid.nc"

lig_prmtop = f"{ppi_path}/2.redock/1.amber/2OOB_A:B/ligand.prmtop"
lig_inpcrd = f"{ppi_path}/2.redock/1.amber/2OOB_A:B/ligand.inpcrd"

rho = 9.0
rc_scale = 0.76
rs_scale = 0.53
rm_scale = 0.55
lc_scale = 0.81
ls_scale = 0.50
lm_scale = 0.54


def _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, grid_nc_file):
    rec_grid = RecGrid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, 
                        grid_nc_file, new_calculation=False)
    return rec_grid

def _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid):
    lig_grid = LigGrid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid)
    return lig_grid

rec_grid = _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, grid_nc_file)

lig_grid = _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid)

/home/jtufts/src/p39/fftanalysis
[1.]
/home/jtufts/Desktop/FFT_PPI/2.redock/4.receptor_grid/2OOB_A:B/grid.nc
[  0.    0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.    5.5
   6.    6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5
  12.   12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5
  18.   18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5
  24.   24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5
  30.   30.5  31.   31.5  32.   32.5  33.   33.5  34.   34.5  35.   35.5
  36.   36.5  37.   37.5  38.   38.5  39.   39.5  40.   40.5  41.   41.5
  42.   42.5  43.   43.5  44.   44.5  45.   45.5  46.   46.5  47.   47.5
  48.   48.5  49.   49.5  50.   50.5  51.   51.5  52.   52.5  53.   53.5
  54.   54.5  55.   55.5  56.   56.5  57.   57.5  58.   58.5  59.   59.5
  60.   60.5  61.   61.5  62.   62.5  63.   63.5  64.   64.5  65.   65.5
  66.   66.5  67.   67.5  68.   68.5  69.   69.5  70.   70.5  71.   71.5
  72.   72.5  7

In [4]:
names = ["SASA"]
lgrid = lig_grid.get_ligand_grids(names, [0,0,0])

In [8]:
import numpy as np
rgrid = np.add(sasar, sasai*1.j)

In [8]:
print((lgrid["SASA"]*rgrid).sum())

0j


In [9]:
lgrid_trans = grids.translate_grid(lgrid["SASA"], np.array([68,87,45]))

In [10]:
print((lgrid_trans*rgrid).sum())

(551+0j)


In [9]:
# fft test
lfft = np.fft.fftn(lgrid["SASA"].conjugate())
rfft = np.fft.fftn(rgrid)
cfft = np.fft.ifftn(rfft*lfft.conjugate())
score = np.real(cfft) - np.imag(cfft)

In [34]:
for i in range(9):
    # rx,ry,rz = np.random.randint(0, counts[0], 3)
    # rx = np.random.randint(19, 39, 1)
    # ry = np.random.randint(81, 101, 1)
    # rz = np.random.randint(74, 94, 1)
    rx = np.random.randint(26, 32, 1)
    ry = np.random.randint(113, 119, 1)
    rz = np.random.randint(49, 55, 1)
    lgrid_trans = grids.translate_grid(lgrid["SASA"], [rx,ry,rz])
    result = (lgrid_trans*rgrid).sum()
    print((rx,ry,rz), result, cfft[rx,ry,rz], (np.real(result)-np.imag(result)), score[rx,ry,rz])

(array([29]), array([115]), array([49])) (1324+432j) [1324.+432.j] 892.0 [892.]
(array([30]), array([115]), array([53])) (1523+837j) [1523.+837.j] 686.0 [686.]
(array([26]), array([117]), array([54])) (1295+504j) [1295.+504.j] 791.0 [791.]
(array([28]), array([114]), array([51])) (1426+738j) [1426.+738.j] 688.0 [688.]
(array([27]), array([113]), array([49])) (1281+990j) [1281.+990.j] 291.0 [291.]
(array([31]), array([114]), array([49])) (1395+756j) [1395.+756.j] 639.0 [639.]
(array([28]), array([116]), array([52])) (1363+234j) [1363.+234.j] 1129.0 [1129.]
(array([30]), array([117]), array([50])) (1251+216j) [1251.+216.j] 1035.0 [1035.]
(array([31]), array([114]), array([54])) (1584+1728j) [1584.+1728.j] -144.0 [-144.]


In [32]:
test = np.where(cfft == np.amax(cfft))

test2 = np.where(score == np.amax(score))

print(np.amax(cfft), test)
print(np.amax(score), test2)


(4354.999999999999+9981j) (array([53]), array([91]), array([84]))
1246.0 (array([29]), array([116]), array([52]))


In [37]:
rec_grid._crd = np.concatenate((rec_grid._crd, lig_grid._crd)) 

In [40]:
import copy
test1 = copy.deepcopy(rec_grid)

In [43]:
print(test1, rec_grid)

<bpmfwfft.grids.RecGrid object at 0x7fa55bde16d0> <bpmfwfft.grids.RecGrid object at 0x7fa55e4703a0>


In [44]:
rec_grid._prmtop.keys()

dict_keys(['CHARGE_E_UNIT', 'R_LJ_CHARGE', 'A_LJ_CHARGE', 'LJ_SIGMA', 'LJ_EPSILON', 'VDW_RADII', 'POINTERS', 'MASS', 'PDB_TEMPLATE'])

In [5]:
keys = list(rec_grid._prmtop.keys())

import copy
complex_grid = copy.deepcopy(rec_grid)
for key in keys:
    if isinstance(complex_grid._prmtop[key], np.ndarray):
        print(key, complex_grid._prmtop[key].shape)
        complex_grid._prmtop[key] = np.concatenate((rec_grid._prmtop[key],lig_grid._prmtop[key]))
    else:
        subkeys = list(rec_grid._prmtop[key].keys())
        for subkey in subkeys:
            if isinstance(complex_grid._prmtop[key][subkey], np.ndarray):
                print(key, subkey, complex_grid._prmtop[key][subkey].shape)
                complex_grid._prmtop[key][subkey] = np.concatenate((rec_grid._prmtop[key][subkey],lig_grid._prmtop[key][subkey]))
            else:
                subkeys2 = list(rec_grid._prmtop[key][subkey].keys())
                for subkey2 in subkeys2:
                    if isinstance(complex_grid._prmtop[key][subkey], np.ndarray):
                        print(key, subkey, subkey2, complex_grid._prmtop[key][subkey][subkey2].shape)
                        complex_grid._prmtop[key][subkey][subkey2] = np.concatenate((rec_grid._prmtop[key][subkey][subkey2],lig_grid._prmtop[key][subkey][subkey2]))
                
                
print(rec_grid._prmtop["VDW_RADII"].shape(), complex_grid._prmtop["VDW_RADII"])

CHARGE_E_UNIT (1231,)
R_LJ_CHARGE (1231,)
A_LJ_CHARGE (1231,)
LJ_SIGMA (1231,)
LJ_EPSILON (1231,)
VDW_RADII (1231,)


AttributeError: 'numpy.int64' object has no attribute 'keys'

In [50]:
print(type(rec_grid._prmtop["VDW_RADII"]))

<class 'numpy.ndarray'>


In [5]:
print(rec_grid._)

AttributeError: 'RecGrid' object has no attribute 'keys'

In [4]:
import socket
print(socket.gethostname())

jim-Mint


In [11]:
def cat_grids(grid1, grid2):
    prmtop = cat_dictionaries(grid1._prmtop,grid2._prmtop)
    crd = cat_dictionaries(grid1._crd,grid2._crd)
    cat_grid = copy.deepcopy(grid1)
    cat_grid._prmtop = prmtop
    cat_grid._crd = crd
    return cat_grid

def cat_dictionaries(dict1, dict2):
    dict1_copy = copy.deepcopy(dict1)
    dict2_copy = copy.deepcopy(dict2)
    
    if isinstance(dict1_copy, dict):
        keys = list(dict1_copy.keys())
        for key in keys:
            # print(keys)
            # print(key)
            dict1_copy[key] = cat_values(dict1[key], dict2[key])
        return dict1_copy
    
    elif isinstance(dict1_copy, np.ndarray):
        return np.concatenate((dict1_copy,dict2_copy))

def cat_values(a1, a2):
    array1 = copy.deepcopy(a1)
    array2 = copy.deepcopy(a2)
    if isinstance(array1, np.ndarray):
        return np.concatenate((array1,array2))
    elif isinstance(array1, dict):
        return cat_dictionaries(array1, array2)
    elif isinstance(array1, (int, np.int64)):
        # print(f"{array1+array2}")
        return array1 + array2
    elif isinstance(array1, list):
        # print(type(array1))
        # print(f'array1:{len(array1)}, array2:{len(array2)}')
        array1.extend(array2)
        # print(f'complex:{len(array1)}, sum:{len(a1)+len(a2)}')
        return array1

        
        
import copy
# result = map(cat_dictionaries, rec_grid._prmtop, lig_grid._prmtop)
vector = np.array([29,116,52])*lig_grid._spacing
lig_grid._move_ligand_to_lower_corner()
print(vector)
lig_grid.translate_ligand(vector)
com_grid = cat_grids(rec_grid, lig_grid)
# result1 = cat_dictionaries(rec_grid._prmtop, lig_grid._prmtop)
# result2 = cat_dictionaries(rec_grid._crd,lig_grid._crd)
# com_grid = copy.deepcopy(rec_grid)
# com_grid._prmtop = result1
# com_grid._crd = result2

Ligand translated by [-14.5 -58.  -26. ]
[1.0249821 1.0339973 1.121    ]
[14.5 58.  26. ]


In [12]:
com_grid.write_pdb('/home/jtufts/Desktop/test.pdb', "w")
com_grid.write_box('/home/jtufts/Desktop/testbox.pdb')

NATOM:2008


In [19]:
# print(cat_dictionaries(rec_grid._prmtop['POINTERS'], lig_grid._prmtop['POINTERS']))
print(com_grid._get_molecule_sasa(0.14, 960).sum())
print(lig_grid._get_molecule_sasa(0.14, 960).sum())
print(rec_grid._get_molecule_sasa(0.14, 960).sum())
print(rec_grid._get_molecule_sasa(0.14, 960).sum()+lig_grid._get_molecule_sasa(0.14, 960).sum())

7170.8086
3989.5044
4483.7715
8473.275


In [7]:
print(com_grid._prmtop)

{'CHARGE_E_UNIT': array([ 0.1592,  0.1984,  0.1984, ...,  0.6631, -0.7697, -0.7697]), 'R_LJ_CHARGE': array([9.71747515e+02, 3.74142723e-01, 3.74142723e-01, ...,
       9.05522867e+02, 6.16341138e+02, 6.16341138e+02]), 'A_LJ_CHARGE': array([20.01653728,  0.21651778,  0.21651778, ..., 16.29574889,
       16.80604034, 16.80604034]), 'LJ_SIGMA': array([3.24999852, 1.06907846, 1.06907846, ..., 3.39966951, 2.9599219 ,
       2.9599219 ]), 'LJ_EPSILON': array([0.17  , 0.0157, 0.0157, ..., 0.086 , 0.21  , 0.21  ]), 'VDW_RADII': array([1.55, 1.2 , 1.2 , ..., 1.7 , 1.52, 1.52]), 'POINTERS': {'NATOM': 2008, 'NTYPES': 26, 'NBONH': 1018, 'MBONA': 1001, 'NTHETH': 2329, 'MTHETA': 1347, 'NPHIH': 4601, 'MPHIA': 4278, 'NHPARM': 0, 'NPARM': 0, 'NNB': 11102, 'NRES': 126, 'NBONA': 1001, 'NTHETA': 1347, 'NPHIA': 4278, 'NUMBND': 101, 'NUMANG': 238, 'NPTRA': 298, 'NATYP': 49, 'NPHB': 0, 'IFPERT': 0, 'NBPER': 0, 'NGPER': 0, 'NDPER': 0, 'MBPER': 0, 'MGPER': 0, 'MDPER': 0, 'IFBOX': 0, 'NMXRS': 48, 'IFCAP': 0, 'N

In [61]:
print(len(result['PDB_TEMPLATE']['RES_ORDER']))
print(len(result['PDB_TEMPLATE']['ATOM_NAME']))
print(len(result['PDB_TEMPLATE']['RES_NAME']))
print(len(lig_grid._prmtop['PDB_TEMPLATE']['RES_NAME']))
print(len(rec_grid._prmtop['PDB_TEMPLATE']['RES_NAME']))
print(len(rec_grid._prmtop['PDB_TEMPLATE']['RES_NAME'])+len(lig_grid._prmtop['PDB_TEMPLATE']['RES_NAME']))

5117
5117
5117
777
5117
5894
